# Chart generations Notebook

## Import Libraries

In [1]:
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as offline
import os
import numpy as np
from plotly.offline import plot

In [2]:
offline.init_notebook_mode(connected=True)

## Bubble Chart for Emissions Intensity


In [3]:
corp_name = "Alphabet Inc. / Google"

# Step 1: Set up the data sources

In [4]:
XLSX_PATH = os.path.join ('../django_project/static/django_project', 'data', 'sp100_data.xlsx')
COLS_TO_USE = {
    'companies':['company_id', 'company_name'],
    'GHG19':['company_id', 'gross_total_scope1', 'gross_loc_scope2', 'gross_mkt_scope2'],
    'grouping':['company_id', 'Sector1'],
    'financials':['company_id', 'Revenue']
}


# Step 2: Merge the tables

In [5]:
all_df = []
for sheetname, cols in COLS_TO_USE.items():
    all_df.append(pd.read_excel(
        XLSX_PATH, 
        sheet_name = sheetname,
        engine = 'openpyxl', 
        usecols = cols
        ))
#all_data.to_pickle (os.path.join ('..','data_frame.pickle'))

In [6]:
merged_df = all_df[0]
for i in range(len(COLS_TO_USE) - 1):
    merged_df = pd.merge(
        left = merged_df,
        right = all_df[i+1],
        how="left",
        on="company_id"
    )

In [7]:
merged_df.head()

,company_id,company_name,gross_total_scope1,gross_loc_scope2,gross_mkt_scope2,Sector1,Revenue
0,1,3M,4050000,1780000,1320000,Industrials,3.213600e+10
1,2,Abbott Laboratories,533000,518000,439000,Healthcare,3.190400e+10
2,3,AbbVie Inc.,314421,308204,249777,Healthcare,3.326600e+10
3,4,Accenture,18923,281489,214680,Technology,4.320000e+10
4,5,Adobe Inc.,11816,56113,43526,Technology,1.117100e+10


# Step 3: Filter the sector

In [8]:
a =  merged_df[merged_df['company_name']==corp_name]['Sector1']
if len(a) > 0:
    sector = a.iloc[0]

In [9]:
merged_df_sector = merged_df.loc[merged_df['Sector1']==sector]

# Calculate GHG Intensity

In [10]:
conditions = [
    (merged_df_sector['gross_loc_scope2'] > 0) & (merged_df_sector['gross_mkt_scope2'] > 0),
    (merged_df_sector['gross_loc_scope2'] == 0),
    (merged_df_sector['gross_mkt_scope2'] == 0),
]
values = ['case1', 'case2', 'case3']
merged_df_sector['scope2_cases'] = np.select(conditions, values)
#merged_df_sector['scope2'] = np.where(merged_df_sector['gross_loc_scope2']>0, True, False)

C:\Users\vince\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [11]:
conditions = [
    (merged_df_sector['scope2_cases'] == 'case1'),
    (merged_df_sector['scope2_cases'] == 'case2'),
    (merged_df_sector['scope2_cases'] == 'case3')
]
values = [merged_df_sector[['gross_loc_scope2','gross_mkt_scope2']].min(axis=1), merged_df_sector['gross_mkt_scope2'], merged_df_sector['gross_loc_scope2']]
merged_df_sector['scope2_calc'] = np.select(conditions, values)

C:\Users\vince\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [12]:
merged_df_sector

,company_id,company_name,gross_total_scope1,gross_loc_scope2,gross_mkt_scope2,Sector1,Revenue,scope2_cases,scope2_calc
6,7,Alphabet Inc. / Google,66686,5116949,794267,Communication Services,1.618570e+11,case1,794267
14,15,AT&T Inc.,990955,6015122,5534088,Communication Services,1.812000e+11,case1,5.53409e+06
30,31,Comcast,498455,1568717,1496771,Communication Services,1.089420e+11,case1,1.49677e+06
89,90,The Walt Disney Company,855073,1010307,931544,Communication Services,6.957000e+10,case1,931544
95,96,Verizon Communications,358753,4006874,3982613,Communication Services,1.318600e+11,case1,3.98261e+06


In [13]:
merged_df_sector = merged_df_sector.assign(scope1_plus_scope2= (merged_df_sector['gross_total_scope1'] + merged_df_sector['scope2_calc']))

In [14]:
merged_df_sector.assign(intensity=merged_df_sector['scope1_plus_scope2'] / merged_df_sector['Revenue'])

,company_id,company_name,gross_total_scope1,gross_loc_scope2,gross_mkt_scope2,Sector1,Revenue,scope2_cases,scope2_calc,scope1_plus_scope2,intensity
6,7,Alphabet Inc. / Google,66686,5116949,794267,Communication Services,1.618570e+11,case1,794267,860953,5.31922e-06
14,15,AT&T Inc.,990955,6015122,5534088,Communication Services,1.812000e+11,case1,5.53409e+06,6.52504e+06,3.60102e-05
30,31,Comcast,498455,1568717,1496771,Communication Services,1.089420e+11,case1,1.49677e+06,1.99523e+06,1.83146e-05
89,90,The Walt Disney Company,855073,1010307,931544,Communication Services,6.957000e+10,case1,931544,1.78662e+06,2.56809e-05
95,96,Verizon Communications,358753,4006874,3982613,Communication Services,1.318600e+11,case1,3.98261e+06,4.34137e+06,3.29241e-05


# Fetch specific values for the selected company

In [15]:
corp_x = merged_df_sector[merged_df_sector['company_name']==corp_name]['scope1_plus_scope2'].iloc[0]
corp_y = merged_df_sector[merged_df_sector['company_name']==corp_name]['Revenue'].iloc[0]

# Fetch data series for the bubble chart data

In [16]:
x0 = merged_df_sector['scope1_plus_scope2']
y0 = merged_df_sector['Revenue']


In [17]:
"""
upper_left_ann = dict (xref="x domain",
                       yref="paper",
                       x=0.10,
                       y=1.10,
                       text="<b>Corporate Behemoths</b> <br> High Revenue, High Emissions",
                       showarrow = False,
                       bgcolor = 'blue',
                       font = {'color':'white'},
                       opacity = 0.5
                      )
upper_right_ann = dict (xref="x domain",
                       yref="paper",
                       x=0.90,
                       y=1.10,
                       text="<b>Sustainability Leaders</b> <br> High Revenue, Low Emissions",
                       showarrow = False,
                       bgcolor = 'blue',
                       font = {'color':'white'},
                       opacity = 0.5
                       )
lower_right_ann = dict (xref="x domain",
                       yref="paper",
                       x=0.90,
                       y=-0.15,
                       text="<b>Small Players</b> <br> Low Revenue, Low Emissions",
                       showarrow = False,
                       bgcolor = 'blue',
                       font = {'color':'white'},
                       opacity = 0.5
                       )
lower_left_ann = dict (xref="x domain",
                       yref="paper",
                       x=0.10,
                       y=-0.15,
                       text="<b>Worst Offenders</b> <br> Low Revenue, High Emissions",
                       showarrow = False,
                      bgcolor = 'blue',
                       font = {'color':'white'},
                       opacity = 0.5
                      )
"""

'\nupper_left_ann = dict (xref="x domain",\n                       yref="paper",\n                       x=0.10,\n                       y=1.10,\n                       text="<b>Corporate Behemoths</b> <br> High Revenue, High Emissions",\n                       showarrow = False,\n                       bgcolor = \'blue\',\n                       font = {\'color\':\'white\'},\n                       opacity = 0.5\n                      )\nupper_right_ann = dict (xref="x domain",\n                       yref="paper",\n                       x=0.90,\n                       y=1.10,\n                       text="<b>Sustainability Leaders</b> <br> High Revenue, Low Emissions",\n                       showarrow = False,\n                       bgcolor = \'blue\',\n                       font = {\'color\':\'white\'},\n                       opacity = 0.5\n                       )\nlower_right_ann = dict (xref="x domain",\n                       yref="paper",\n                       x=0.90,\n 

In [54]:
layout = go.Layout (
    title = '<b>GHG Emissions vs. Revenue</b><br>Sector: ' + sector,
    title_x = 0.5,
    titlefont = dict(
        family = 'Arial',
        size = 16),
    plot_bgcolor = 'antiquewhite',
    xaxis =  dict(
        autorange = "reversed",
        type = 'log',
        title = '<i>high</i>----------<b>Scope1+2 Emissions</b>----------<i>low</i>'),
    yaxis = dict(
        type = 'log',
        title = 'Revenue'),
    #annotations = [upper_left_ann, upper_right_ann, lower_left_ann, lower_right_ann]
)

In [55]:
x_median_x = [x0.min(), x0.max()]
y_median_x = [y0.median(), y0.median()]
x_median_y = [x0.median(), x0.median()]
y_median_y = [y0.min(), y0.max()]

In [56]:
index_corp = merged_df_sector.loc[merged_df_sector['company_name'] == corp_name].index
# Delete these row indexes from dataFrame
merged_df_sector.drop(index_corp , inplace=True)

In [57]:
merged_df_sector

,company_id,company_name,gross_total_scope1,gross_loc_scope2,gross_mkt_scope2,Sector1,Revenue,scope2_cases,scope2_calc,scope1_plus_scope2
14,15,AT&T Inc.,990955,6015122,5534088,Communication Services,1.812000e+11,case1,5.53409e+06,6.52504e+06
30,31,Comcast,498455,1568717,1496771,Communication Services,1.089420e+11,case1,1.49677e+06,1.99523e+06
89,90,The Walt Disney Company,855073,1010307,931544,Communication Services,6.957000e+10,case1,931544,1.78662e+06
95,96,Verizon Communications,358753,4006874,3982613,Communication Services,1.318600e+11,case1,3.98261e+06,4.34137e+06


In [58]:
index_corp

Int64Index([], dtype='int64')

In [59]:
trace1 = go.Scatter(x=x_median_y, 
                    y=y_median_y, 
                    showlegend = False, 
                    name='Median y',
                    mode = "lines",
                    line = dict(color='gray', 
                                width=2, 
                                dash='dash'))

trace2 = go.Scatter(x=x_median_x, 
                         y=y_median_x, 
                         showlegend = False, 
                         name='Median x', 
                         mode = "lines",
                         line = {'color':'gray', 
                                 'width':2, 
                                 'dash':'dash'}
                        )

trace3 = go.Scatter(
    x=merged_df_sector['scope1_plus_scope2'], 
    y=merged_df_sector['Revenue'],
    showlegend = False,
    name="benchmark", 
    text=merged_df_sector['company_name'],
    mode = 'markers+text',
    line = {'color':'black'}, 
    marker = {
        'color': '#488f31',
        'opacity': 0.4,
        'size':12
    },
    textposition='top center'
    )
trace4 = go.Scatter(
    x=[corp_x], 
    y=[corp_y],
    showlegend = False,
    name=corp_name, 
    text= corp_name,
    textfont=dict(
        size=12,
        color="#de425b"),
    mode = 'markers+text',
    line = {'color':'#de425b'}, 
    marker = {
        'color': '#de425b',
        'opacity': 1,
        'size':20
    },
    textposition='top center'
    )

fig = go.Figure(data = [trace1, trace2, trace3, trace4], 
                layout = layout)
                
fig.update_layout(
    autosize=False,
    width=500,
    height=500,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=80,
        pad=4
    ),
    paper_bgcolor="#bad0af",
    plot_bgcolor = '#f1f1f1',
)
config = {'displaylogo': False}


In [60]:
offline.iplot(fig, config = config)

In [32]:
name_fig = "bubble_intensity_{}".format(corp_name)
fig.write_image("../django_project/static/django_project/images/charts/{}.svg".format(name_fig), scale=3, height = 300)
plot(fig, config=config, filename = '../django_project/static/django_project/images/html_exports/{}.html'.format(name_fig), auto_open=False)

FileNotFoundError: [Errno 2] No such file or directory: '../django_project/static/django_project/images/charts/bubble_intensity_Alphabet Inc. / Google.svg'